In [1]:
#imports and scraping raw data from site
#df is the actual betting line
#df2 has other info, mostly using to get player id to name dict
from scipy.stats import poisson
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import datetime as dt
import random
import time
#time.sleep(2.5*60*60)

%run functions.ipynb
%run underdog_scrape.ipynb
%run pp_scrape.ipynb
#%run parlayplay.ipynb

C:\Users\Shared.Westover\AppData\Local\Temp\ipykernel_29216\3263823425.py:10: DeprecationWarning: AppURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  opener = AppURLopener()


200


NotImplementedError: 

NotImplementedError: 

In [2]:
import tls_client
import gzip

headers = {
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "accept-language": "en-US,en;q=0.9",
    "cache-control": "no-cache",
    "pragma": "no-cache",
    "sec-ch-ua": '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "none",
    "sec-fetch-user": "?1",
    "upgrade-insecure-requests": "1",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
}

requests = tls_client.Session(
    client_identifier="chrome112",
)
unabated_url = 'https://content.unabated.com/markets/b_playerprops.json?'
response1 = requests.get(unabated_url)

data = response1.json()

200


In [3]:
#team from id dict
teamtemp = pd.json_normalize(data['teams'],max_level=0)
teams = pd.DataFrame()
for league_code in range(len(teamtemp.T)):
    teams = pd.concat((teams,pd.json_normalize(teamtemp.iloc[0,league_code])))
teams = teams[['abbreviation','id']]
teamdict = dict(zip(teams.id,teams.abbreviation))

In [4]:
#bookie from id dict
bookies = pd.json_normalize(data['marketSources'])
bookies
id2book = dict(list(zip(bookies.id,bookies.name)))
id2book

{68: 'Circa - Delayed',
 70: 'Pinnacle - 3838',
 71: 'CaesarsDirect',
 58: 'Pinnacle - Delayed',
 57: 'FanDuel - Delayed',
 72: 'PrizePicks',
 73: 'Underdog Fantasy',
 74: 'SugarHouse - Direct',
 75: 'BetRivers - Direct',
 76: 'Unibet - Direct',
 15: 'BetAmerica',
 16: 'Betfred',
 64: 'Circa Pool',
 40: 'BetWay',
 19: 'BetWildwood',
 18: 'Borgata',
 34: 'Elite Sportsbook',
 41: 'FireKeepers',
 42: 'Fubo',
 43: 'GamBetDC',
 23: 'Golden Nugget',
 44: 'Gun Lake',
 24: 'Hard Rock',
 45: 'MaximBet',
 35: 'Play Maverick Sports',
 39: 'PlayUp',
 26: 'Resorts',
 46: 'SBK',
 47: 'SI',
 29: 'Tipico',
 30: 'TwinSpires',
 5: 'William Hill',
 32: 'Wind Creek',
 61: 'OfficeFootballPool',
 62: 'RunYourPool',
 63: 'DraftKings Pool',
 2: 'FanDuel',
 7: 'Pinnacle',
 14: 'Unused (used to be Bet365)',
 13: '888sports',
 51: '5Dimes',
 12: 'Action24/7',
 38: 'BallyBet',
 50: 'Retail Consensus',
 21: 'FoxBet',
 33: 'WynnBet',
 20: 'Caesars',
 28: 'SuperBook',
 8: 'Bookmaker',
 49: 'Unabated',
 59: 'Buckeye'

In [5]:
data['propsPeopleEvents'].keys()

dict_keys(['lg5:pt1:pregame', 'lg2:pt1:pregame', 'lg1:pt1:pregame', 'lg7:pt1:pregame'])

In [6]:
#player json normalize
people_raw = pd.json_normalize(data['people'],max_level=0)
ppl = pd.DataFrame()
for league_code in range(len(people_raw.T)):
    y = pd.json_normalize(people_raw.T.iloc[league_code])
    ppl = pd.concat((ppl,y))
ppl.sample(3)

,headshotUrl,firstName,lastName,preferredName,jerseyNumber,country,priorityType,height,weight,position,birthDate,leagueId,team,createdOn,createdBy,modifiedOn,modifiedBy,id
0,None,Myles,Price,None,1,"The Colony, TX, USA",0,70,190,WR,0001-01-01T00:00:00+00:00,2,None,2022-09-11T03:48:01.122306,-28,2023-10-05T00:00:44.768335,-28,61890
0,None,Ladd,McConkey,None,84,"Chatsworth, GA, USA",0,72,185,WR,0001-01-01T00:00:00+00:00,2,None,2022-09-11T03:50:15.298075,-28,2023-10-05T00:00:47.884449,-28,141572
0,players/nfl/35540.png,Parris,Campbell,None,0,"Akron, OH, USA",0,73,205,WR,0001-01-01T00:00:00+00:00,1,None,2022-09-11T03:34:11.74106,-28,2023-10-05T00:00:12.480517,-28,35540


In [7]:
i2p = dict(zip(ppl.id,(ppl.firstName+' '+ppl.lastName)))

In [8]:
#step 1 of json normalize for lines
df = pd.json_normalize(data['propsPeopleEvents'],max_level=0)
lines_df = pd.json_normalize(df.iloc[0,0])

lines_df = pd.DataFrame()
for league_code in df.columns:
    df_temp = pd.json_normalize(df.loc[0,league_code],max_level=0)
    df_temp['league_id'] = league_code.split(':')[0].split('g')[-1]
    lines_df = pd.concat((lines_df,df_temp)).reset_index(drop=True)
lines_df

,eventId,eventStart,teamId,personId,playerPosition,eventTeams,propsMarketSourcesLines,league_id
0,53554,2023-10-06T00:08:00,47,42413,IF,"{'0': {'id': 47, 'rotationNumber': None, 'scor...",{'si0:ms8:an0': {'bt19': {'marketLineId': 8754...,5
1,53554,2023-10-06T00:08:00,53,39182,IF,"{'0': {'id': 47, 'rotationNumber': None, 'scor...",{'si0:ms8:an0': {'bt19': {'marketLineId': 8754...,5
2,53554,2023-10-06T00:08:00,47,5544,OF,"{'0': {'id': 47, 'rotationNumber': None, 'scor...",{'si0:ms8:an0': {'bt19': {'marketLineId': 8754...,5
3,53554,2023-10-06T00:08:00,53,41564,DH,"{'0': {'id': 47, 'rotationNumber': None, 'scor...",{'si0:ms8:an0': {'bt19': {'marketLineId': 8754...,5
4,53554,2023-10-06T00:08:00,53,40714,IF,"{'0': {'id': 47, 'rotationNumber': None, 'scor...",{'si0:ms8:an0': {'bt19': {'marketLineId': 8754...,5
...,...,...,...,...,...,...,...,...
619,53430,2023-10-08T19:00:00,1523,410231,F,"{'0': {'id': 1523, 'rotationNumber': None, 'sc...",{'si0:ms72:an0': {'bt75': {'marketLineId': 876...,7
620,53430,2023-10-08T19:00:00,1523,410233,G,"{'0': {'id': 1523, 'rotationNumber': None, 'sc...",{'si0:ms72:an0': {'bt74': {'marketLineId': 876...,7
621,53430,2023-10-08T19:00:00,1526,409927,G,"{'0': {'id': 1523, 'rotationNumber': None, 'sc...",{'si0:ms72:an0': {'bt75': {'marketLineId': 876...,7
622,53430,2023-10-08T19:00:00,1526,410285,G,"{'0': {'id': 1523, 'rotationNumber': None, 'sc...",{'si0:ms72:an0': {'bt76': {'marketLineId': 876...,7


In [9]:
#replace player id with name
lines_df['person_name'] = lines_df.personId.apply(lambda x: i2p[x])

In [10]:
#replacing team id with team name
teams = pd.json_normalize(lines_df.eventTeams)
teams[['1.id','0.id']]
teams['Team'] =teams['0.id'].apply(lambda x: teamdict[x])
teams['Opp'] =teams['1.id'].apply(lambda x: teamdict[x])
teams = teams[['Team','Opp']]
teams['Opp'][0]

'PHI'

In [11]:
lines_df

,eventId,eventStart,teamId,personId,playerPosition,eventTeams,propsMarketSourcesLines,league_id,person_name
0,53554,2023-10-06T00:08:00,47,42413,IF,"{'0': {'id': 47, 'rotationNumber': None, 'scor...",{'si0:ms8:an0': {'bt19': {'marketLineId': 8754...,5,Luis Arraez
1,53554,2023-10-06T00:08:00,53,39182,IF,"{'0': {'id': 47, 'rotationNumber': None, 'scor...",{'si0:ms8:an0': {'bt19': {'marketLineId': 8754...,5,Trea Turner
2,53554,2023-10-06T00:08:00,47,5544,OF,"{'0': {'id': 47, 'rotationNumber': None, 'scor...",{'si0:ms8:an0': {'bt19': {'marketLineId': 8754...,5,Bryan De La Cruz
3,53554,2023-10-06T00:08:00,53,41564,DH,"{'0': {'id': 47, 'rotationNumber': None, 'scor...",{'si0:ms8:an0': {'bt19': {'marketLineId': 8754...,5,Kyle Schwarber
4,53554,2023-10-06T00:08:00,53,40714,IF,"{'0': {'id': 47, 'rotationNumber': None, 'scor...",{'si0:ms8:an0': {'bt19': {'marketLineId': 8754...,5,Bryce Harper
...,...,...,...,...,...,...,...,...,...
619,53430,2023-10-08T19:00:00,1523,410231,F,"{'0': {'id': 1523, 'rotationNumber': None, 'sc...",{'si0:ms72:an0': {'bt75': {'marketLineId': 876...,7,Jonquel Jones
620,53430,2023-10-08T19:00:00,1523,410233,G,"{'0': {'id': 1523, 'rotationNumber': None, 'sc...",{'si0:ms72:an0': {'bt74': {'marketLineId': 876...,7,Courtney Vandersloot
621,53430,2023-10-08T19:00:00,1526,409927,G,"{'0': {'id': 1523, 'rotationNumber': None, 'sc...",{'si0:ms72:an0': {'bt75': {'marketLineId': 876...,7,Jackie Young
622,53430,2023-10-08T19:00:00,1526,410285,G,"{'0': {'id': 1523, 'rotationNumber': None, 'sc...",{'si0:ms72:an0': {'bt76': {'marketLineId': 876...,7,Kelsey Plum


In [12]:
#renaming stats
bt2stat = {'bt77': 'TRB',
          'bt73': 'PTS',
          'bt69': '3P',
          'bt78': 'TRB+AST',
          'bt74': 'PTS+AST',
          'bt75': 'PTS+TRB',
           'bt76': 'PTS+TRB+AST',
           'bt70': 'AST',
           'bt84': 'TO',
           'bt82': 'BLK+STL',
           'bt81': 'STL',
           'bt71': 'BLK',
           'bt72': 'Techs',
          'bt83': 'Triple Double',
           'bt77': 'TRB',
          'bt73': 'PTS',
          'bt69': '3P',
          'bt78': 'TRB + AST',
          'bt74': 'PTS+AST',
          'bt75': 'PTS+TRB',
           'bt76': 'PTS+TRB+AST',
           'bt70': 'AST',
           'bt84': 'TO',
           'bt82': 'BLK + STL',
           'bt81': 'STL',
           'bt71': 'BLK',
           'bt72': 'Techs',
          'bt83': 'Triple Double',
          'bt85': 'Total Hits',
          'bt19': 'Total Bases',
          'bt17': 'Pitcher Strikeouts',
          'bt18': 'Home Runs'}

In [14]:
#%%timeit

#func to further normalize and aggregate lines for each player
#biggest bottleneck speed wise
def get_odds(row):
    side_dict = {'si0':'over',
                 'si1':'under'}
    event_time = lines_df.eventStart[row]
    person_id = lines_df.personId[row]
    person_name = lines_df.person_name[row]
    league = lines_df.league_id[row]
    player_json = lines_df['propsMarketSourcesLines'][row]
    player_keys = player_json.keys()
    test_list = []
    for key in player_keys: # key is something like 'si1:ms8:an0'  where the first chunk is 1 or 0 meaning over or under, middle chunk is bookie id
        for stat_id in player_json[key]:
            book = key.split(':')[1][2:]
            book_name = id2book.get(int(book), 'idk')
            side = np.where(key.split(':')[0] == 'si0', 'over', 'under')
            side = side_dict.get(key.split(':')[0],'N/A')
            stat = bt2stat.get(stat_id, stat_id)
            price = player_json[key][stat_id]['americanPrice']
            points = player_json[key][stat_id]['points']
            test_list.append([points,price,side,stat,book_name,league,event_time])
             # points, price, side, stat, book
    results_df = pd.DataFrame(test_list)
    results_df.columns = ['points','price','side','stat','book','league_id','event_time']
    results_df['player'] = person_name
    results_df['player_id'] = person_id
    results_df = results_df[['player',"player_id",'points','price','side','stat','book','league_id','event_time']].dropna()
    results_df['Prob'] = results_df['price'].apply(lambda x: (-x/(100-x)) if x < 0 else 100/(x+100))
    results_df['Opp'] = teams['Opp'][row]
    results_df['Team'] = teams['Team'][row]
    return results_df
luka = get_odds(1)
luka.head()

,player,player_id,points,price,side,stat,book,league_id,event_time,Prob,Opp,Team
2,Trea Turner,39182,0.5,-196.0,over,Total Hits,Caesars,5,2023-10-06T00:08:00,0.662162,PHI,MIA
3,Trea Turner,39182,0.5,450.0,over,Home Runs,Caesars,5,2023-10-06T00:08:00,0.181818,PHI,MIA
4,Trea Turner,39182,1.5,130.0,over,Total Bases,Caesars,5,2023-10-06T00:08:00,0.434783,PHI,MIA
5,Trea Turner,39182,0.5,-190.0,over,Total Hits,BetMGM,5,2023-10-06T00:08:00,0.655172,PHI,MIA
6,Trea Turner,39182,0.5,400.0,over,Home Runs,BetMGM,5,2023-10-06T00:08:00,0.200000,PHI,MIA


In [15]:
ppl.loc[ppl.id == 41561]

,headshotUrl,firstName,lastName,preferredName,jerseyNumber,country,priorityType,height,weight,position,birthDate,leagueId,team,createdOn,createdBy,modifiedOn,modifiedBy,id


In [16]:
today = dt.datetime.now().replace(microsecond=0,second=0)
print(f'Lines/unabated_raw_{today.year}_{today.month}_{today.day}.csv')

Lines/unabated_raw_2023_10_5.csv


In [17]:
#getting lines for every player
odds = pd.DataFrame()
for i in range(len(lines_df)):
    if i % 20==0:
        print(i,'/',len(lines_df))
    odds = pd.concat((odds,get_odds(i)))
odds.sample(3)

0 / 624
20 / 624
40 / 624
60 / 624
80 / 624
100 / 624
120 / 624
140 / 624
160 / 624
180 / 624
200 / 624
220 / 624
240 / 624
260 / 624
280 / 624
300 / 624
320 / 624
340 / 624
360 / 624
380 / 624
400 / 624
420 / 624
440 / 624
460 / 624
480 / 624
500 / 624
520 / 624
540 / 624
560 / 624
580 / 624
600 / 624
620 / 624


,player,player_id,points,price,side,stat,book,league_id,event_time,Prob,Opp,Team
1,Josh McCray,226411,46.5,-114.0,over,bt12,SportsBetting,2,2023-10-07T00:00:00,0.532710,ILL,NEB
54,Kyle Pitts,35678,28.5,-110.0,over,bt16,BetOnline,1,2023-10-08T17:00:00,0.523810,ATL,HOU
111,D'Andre Swift,38710,13.5,-109.0,over,bt16,Unibet,1,2023-10-08T20:05:00,0.521531,LAR,PHI


In [18]:
odds.to_csv('Lines/unabated_raw')
odds.sample(2)


,player,player_id,points,price,side,stat,book,league_id,event_time,Prob,Opp,Team
2,Terrel Bernard,34293,7.5,-103.0,under,bt57,Caesars,1,2023-10-08T13:30:00,0.507389,BUF,JAX
19,Alexander Mattison,38241,54.5,-125.0,under,bt12,DraftKings,1,2023-10-08T20:25:00,0.555556,MIN,KC


In [19]:
#bookmaker is broken right now
#not sure how to handle Total Bases because it isn't poisson
odds = odds.loc[odds.book !='Bookmaker']
odds = odds.loc[~((odds.stat =='Total Bases') & (odds.points==1.5))]
len(odds)

14876

In [20]:
odds_agg = odds.groupby(['player','player_id','side','points','stat','league_id','event_time','Opp','Team'],as_index=False)['Prob'].agg(func=['mean','count']).reset_index()
odds_agg
#['Player','Stat','Line','o_Prob','u_Prob','num_books','means']

,player,player_id,side,points,stat,league_id,event_time,Opp,Team,mean,count
0,A'ja Wilson,410295,over,1.5,AST,7,2023-10-08T19:00:00,LVA,NYL,0.476190,1
1,A'ja Wilson,410295,over,9.5,TRB,7,2023-10-08T19:00:00,LVA,NYL,0.595085,7
2,A'ja Wilson,410295,over,10.5,TRB,7,2023-10-08T19:00:00,LVA,NYL,0.531282,4
3,A'ja Wilson,410295,over,23.5,PTS,7,2023-10-08T19:00:00,LVA,NYL,0.555812,11
4,A'ja Wilson,410295,over,34.0,PTS+TRB,7,2023-10-08T19:00:00,LVA,NYL,0.574468,1
...,...,...,...,...,...,...,...,...,...,...,...
4203,Zach Wilson,34241,under,202.5,bt14,1,2023-10-08T20:25:00,DEN,NYJ,0.574468,1
4204,Zach Wilson,34241,under,204.5,bt14,1,2023-10-08T20:25:00,DEN,NYJ,0.543440,6
4205,Zach Wilson,34241,under,216.5,bt64,1,2023-10-08T20:25:00,DEN,NYJ,0.540169,2
4206,Zay Flowers,410752,over,46.5,bt16,1,2023-10-08T17:00:00,PIT,BAL,0.544073,5


In [21]:
odds_condensed = odds_agg.pivot(index=['player', 'player_id', 'points', 'stat', 'league_id', 'event_time', 'Opp', 'Team','count'],
                       columns='side', values='mean').reset_index()
odds_condensed.rename(columns={'over': 'over_prob', 'under': 'under_prob','points':'line'}, inplace=True)
odds_condensed['total'] = odds_condensed['over_prob'] + odds_condensed['under_prob']
odds_condensed['over_prob'] = odds_condensed['over_prob']/odds_condensed['total']
odds_condensed['under_prob'] = odds_condensed['under_prob']/odds_condensed['total']
odds_condensed.drop('total',axis=1,inplace=True)
odds_condensed['mean_pred'] = odds_condensed.apply(lambda row: prob2mean(row['over_prob'], row['line'], 'over'), axis=1)
#Player	Stat	Line	o_Prob	u_Prob	num_books	means
odds_condensed

side,player,player_id,line,stat,league_id,event_time,Opp,Team,count,over_prob,under_prob,mean_pred
0,A'ja Wilson,410295,1.5,AST,7,2023-10-08T19:00:00,LVA,NYL,1,0.449438,0.550562,1.519638
1,A'ja Wilson,410295,9.5,TRB,7,2023-10-08T19:00:00,LVA,NYL,7,0.556314,0.443686,10.112666
2,A'ja Wilson,410295,10.5,TRB,7,2023-10-08T19:00:00,LVA,NYL,4,0.480514,0.519486,10.508882
3,A'ja Wilson,410295,23.5,PTS,7,2023-10-08T19:00:00,LVA,NYL,11,0.513258,0.486742,23.829074
4,A'ja Wilson,410295,34.0,PTS+TRB,7,2023-10-08T19:00:00,LVA,NYL,1,0.500000,0.500000,34.178042
...,...,...,...,...,...,...,...,...,...,...,...,...
2159,Zach Wilson,34241,201.5,bt14,1,2023-10-08T20:25:00,DEN,NYJ,2,0.498019,0.501981,201.630004
2160,Zach Wilson,34241,202.5,bt14,1,2023-10-08T20:25:00,DEN,NYJ,1,0.500000,0.500000,202.701131
2161,Zach Wilson,34241,204.5,bt14,1,2023-10-08T20:25:00,DEN,NYJ,6,0.498891,0.501109,204.661396
2162,Zach Wilson,34241,216.5,bt64,1,2023-10-08T20:25:00,DEN,NYJ,2,0.500000,0.500000,216.702584


In [30]:
def weighted_average(group):
    weighted_sum = (group['mean_pred'] * group['count']).sum()
    total_weight = group['count'].sum()
    return weighted_sum / total_weight

# Group by 'player' and 'stat', and apply the custom function
temp_table = odds_condensed.groupby(['player', 'stat']).apply(weighted_average).reset_index(name='pred')
final_odds = odds_condensed.merge(temp_table,on=['player','stat'])
final_odds.drop('mean_pred',inplace=True,axis=1)
final_odds

,player,player_id,line,stat,league_id,event_time,Opp,Team,count,over_prob,under_prob,pred
0,A'ja Wilson,410295,1.5,AST,7,2023-10-08T19:00:00,LVA,NYL,1,0.449438,0.550562,1.519638
1,A'ja Wilson,410295,9.5,TRB,7,2023-10-08T19:00:00,LVA,NYL,7,0.556314,0.443686,10.256745
2,A'ja Wilson,410295,10.5,TRB,7,2023-10-08T19:00:00,LVA,NYL,4,0.480514,0.519486,10.256745
3,A'ja Wilson,410295,23.5,PTS,7,2023-10-08T19:00:00,LVA,NYL,11,0.513258,0.486742,23.829074
4,A'ja Wilson,410295,34.0,PTS+TRB,7,2023-10-08T19:00:00,LVA,NYL,1,0.500000,0.500000,34.178042
...,...,...,...,...,...,...,...,...,...,...,...,...
2159,Zach Wilson,34241,201.5,bt14,1,2023-10-08T20:25:00,DEN,NYJ,2,0.498019,0.501981,203.769946
2160,Zach Wilson,34241,202.5,bt14,1,2023-10-08T20:25:00,DEN,NYJ,1,0.500000,0.500000,203.769946
2161,Zach Wilson,34241,204.5,bt14,1,2023-10-08T20:25:00,DEN,NYJ,6,0.498891,0.501109,203.769946
2162,Zach Wilson,34241,216.5,bt64,1,2023-10-08T20:25:00,DEN,NYJ,2,0.500000,0.500000,216.702584
